In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [ ]:
df = pd.DataFrame()
error_log = pd.DataFrame()

sitemap = requests.get('https://autoplius.lt/xml_sitemap/index.xml')
soup = BeautifulSoup(sitemap.text, "html.parser")
ads_sitemap = [tag.text for tag in soup.findAll('loc') if 'ann_list' in tag.text]
for sitemap_url in ads_sitemap:
    print(sitemap_url, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    sitemap_link = requests.get(sitemap_url)
    links_soup = BeautifulSoup(sitemap_link.text, "html.parser")
    products = list(set([(tag.find("xhtml:link", {'hreflang': 'en'})['href'], tag.find('lastmod').text, tag.find('priority').text) for tag in links_soup.findAll('url')]))
    items = []
    for i, url in enumerate(products):
        print(f"{i}/{len(products)} | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        r = requests.get(url[0])
        prod_soup = BeautifulSoup(r.text, "html.parser")
        data = {}
        try:
            data['title'] = prod_soup.find(attrs = {'class': 'page-title'}).text.replace('\n', '')
        except:
            data['title'] = ''
        
        try:
            data['price'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'price'}).text)
        except:
            data['price'] = 'nagotiate'
        
        try:
            data['phone'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'seller-phone-number'}).text)
        except:
            data['phone'] = ''
        
        try:
            data['description'] = prod_soup.findAll('div', {'class': 'announcement-description'})[0].text
        except:
            data['description'] = ''
        
        try:
            data['contact_name'] = prod_soup.find('div', {'class': 'seller-contact-name'}).text
        except:
            data['contact_name'] = ''
            
        try:
            data['contact_location'] = re.sub('(\n)|([ ]{2,})', '', prod_soup.find('div', {'class': 'seller-contact-location'}).text)
        except:
            data['contact_location'] = ''
        
        data['sitemap_link'] = sitemap_url
        data['url'] = url
        data['scrape_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        try:
            params_label = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-label'})]
            params_value = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-value'})]
            params = list(zip(params_label, params_value))
            for p in params:
                data[p[0]] = p[1]
        except Exception as e:
            error_log = error_log.append([{'params': e, 'url': url}])
        
        try:
            features = []
            for row in prod_soup.findAll('div', {'class': 'feature-row'}):
                feature_label = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in row.findAll('div', {'class': 'feature-label'})]
                feature_value = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in row.findAll('span', {'class': 'feature-item'})]
                features.append({feature_label[0]: feature_value})

            for row in features:
                data.update(row)
        except Exception as e:
            error_log = error_log.append([{'features': e, 'url': url}])
        try:
            memorized = re.findall('[0-9]+ lankytojai', prod_soup.findAll('div', {'class': 'bookmark-stats-bar'})[0].text)[0]
        except Exception as e:
            memorized = ''
        try:
            update_timestamp = [tag.text for tag in prod_soup.findAll('span', {'class': 'bar-item'}) if 'Atnaujintas' in tag.text][0]
        except Exception as e:
            update_timestamp = ''

        data['memorized'] = memorized
        data['update_timestamp'] = update_timestamp
        
        data['last_updated'] = url[1]
        data['adv_priority'] = url[2]
        
        items.append(data)
        
    df = df.append(items)
    df.to_csv('output.csv', index=False)

https://autoplius.lt/xml_sitemap/ann_list/1.xml 2020-01-23 23:13:33
0/5000 | 2020-01-23 23:13:45
1/5000 | 2020-01-23 23:13:45
2/5000 | 2020-01-23 23:13:46
3/5000 | 2020-01-23 23:13:46
4/5000 | 2020-01-23 23:13:46
5/5000 | 2020-01-23 23:13:46
6/5000 | 2020-01-23 23:13:47
7/5000 | 2020-01-23 23:13:47
8/5000 | 2020-01-23 23:13:47
9/5000 | 2020-01-23 23:13:48
10/5000 | 2020-01-23 23:13:48
11/5000 | 2020-01-23 23:13:48
12/5000 | 2020-01-23 23:13:49
13/5000 | 2020-01-23 23:13:49
14/5000 | 2020-01-23 23:13:50
15/5000 | 2020-01-23 23:13:50
16/5000 | 2020-01-23 23:13:50
17/5000 | 2020-01-23 23:13:51
18/5000 | 2020-01-23 23:13:51
19/5000 | 2020-01-23 23:13:51
